In [1]:
import pandas as pd
from sodapy import Socrata

In [2]:
client = Socrata("data.cityofnewyork.us",
                 "7MystexXhjDtgtn6dZxzE5Kwb",
                 username="sjw.yates@gmail.com",
                 password="d0gg0N@me$")

In [3]:
babies_ept = "25th-nujf"
doggos_ept = "nu7n-tubp"

In [4]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect, Column, Integer, String

In [5]:
engine = create_engine("sqlite:///../db/doggo-db.sqlite")

In [6]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [7]:
class Baby(Base):
    __tablename__ = 'babies'
    
    id = Column(Integer, primary_key=True)
    brth_yr = Column(Integer)
    gndr = Column(String)
    ethcty = Column(String)
    nm = Column(String)
    cnt = Column(Integer)
    rnk = Column(Integer)
    
    def __repr__(self):
        return "<Baby(brth_yr='%s', gndr='%s', ethcty='%s', nm='%s', cnt='%s', rnk='%s')>" % (
                        self.brth_yr, self.gndr, self.ethcty, self.nm, self.cnt, self.rnk)

In [32]:
Base.metadata.create_all(engine)

In [29]:
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
babies = client.get(babies_ept, limit=1000000000)

In [11]:
for baby in babies:
    new_baby = Baby(brth_yr = baby['brth_yr'],
                    gndr = baby['gndr'],
                    ethcty = baby['ethcty'],
                    nm = baby['nm'],
                    cnt = baby['cnt'],
                    rnk = baby['rnk'])
    session.add(new_baby)

In [12]:
session.commit()

In [15]:
class Doggo(Base):
    __tablename__ = 'doggos'
    
    id = Column(Integer, primary_key=True)
    rownumber = Column(Integer)
    animalname = Column(String)
    animalgender = Column(String)
    animalbirth = Column(Integer)
    breedname = Column(String)
    zipcode = Column(String)
    licenseissueddate = Column(String)
    licenseexpireddate = Column(String)
    extract_year = Column(Integer)
    
    def __repr__(self):
        return "<Doggo(rownumber='%s', animalname='%s', animalgender='%s', animalbirth='%s', breedname='%s', zipcode='%s', licenseissueddate='%s', licenseexpireddate='%s', extract_year='%s')>" % (
                        self.rownumber, self.animalname, self.animalgender, self.animalbirth, self.breedname, self.zipcode, self.licenseissueddate, self.licenseexpireddate, self.extract_year)

In [13]:
doggos = client.get(doggos_ept, limit=1000000000)

In [37]:
for doggo in doggos:
    if 'animalgender' not in doggo:
        doggo['animalgender'] = 'None Specified'
    new_doggo = Doggo(rownumber = int(doggo['rownumber']),
                     animalname = doggo['animalname'],
                     animalgender = doggo['animalgender'],
                     animalbirth = int(doggo['animalbirth']),
                     breedname = doggo['breedname'],
                     zipcode = doggo['zipcode'],
                     licenseissueddate = doggo['licenseissueddate'],
                     licenseexpireddate = doggo['licenseexpireddate'],
                     extract_year = int(doggo['extract_year']))
    session.add(new_doggo)

In [38]:
session.commit()